In [ ]:
#!/usr/python3

import tensorflow as tf
import numpy as np
import os,time,math,sys,random
import matplotlib.pyplot as plt
from PIL import Image

# Reducing GPU usage:

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config = config)
tf.compat.v1.keras.backend.set_session(sess)   

# Using TPU

#tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#tf.config.experimental_connect_to_cluster(tpu)
#tf.tpu.experimental.initialize_tpu_system(tpu)
#tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
# - - - - - - - - - - - - - - - - - - - - - - 

# Defining constants:
IMG_SHAPE = (32, 32, 3)
NOISE_SHAPE = (100,)

NOISE = tf.random.normal([1, NOISE_SHAPE[0]])

TRAIN_SIZE = 20000
LR = 0.0002
BR = 0.5
OPTIMIZER = tf.compat.v1.keras.optimizers.Adam(LR,BR)

BATCH_SIZE = 32
HALF_BATCH = int(BATCH_SIZE/2)
N_EPOCHS = 1000000

GENERATED_IMAGE_PATH = "./result_GAN/fake_faces/"
GENERATED_PLOTS_PATH = "./result_GAN/plots/"
GENERATED_MODEL_PATH =  "./result_GAN/models/"
RESULTS_PATH="./result_GAN/"
# - - - - - - - - - - - - - - - - - - - - - - 

In [ ]:
# - - - - - - - - - - - - - - - - - - - - - - 
# Perform validity check on real versus fake images:
def build_discriminator(img_shape, noutput = 1):
    input_img = tf.keras.layers.Input(shape=img_shape)
    
    d = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', name='block1_conv1')(input_img)
    d = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', name='block1_conv2')(d)
    d = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(d)
    
    d = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block2_conv1')(d)
    d = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block2_conv2')(d)
    d = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(d)
    
    d = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block4_conv1')(d)
    d = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block4_conv2')(d)
    d = tf.keras.layers.MaxPooling2D((2, 2), strides=(1, 1), name='block4_pool')(d)
    
    d         = tf.keras.layers.Flatten()(d)
    d         = tf.keras.layers.Dense(1024,      activation="relu")(d)
    out       = tf.keras.layers.Dense(noutput,   activation='sigmoid')(d)
    model     = tf.keras.models.Model(input_img, out)
    model.summary()
    return model
# - - - - - - - - - - - - - - - - - - - - - - 

In [ ]:
# - - - - - - - - - - - - - - - - - - - - - - 
# Generate my fake images:
def build_generator(img_shape, noise_shape = NOISE_SHAPE):
    print(img_shape)
    print("hue")
    input_noise = tf.keras.layers.Input(shape = noise_shape) 
    g = tf.keras.layers.Dense(1024, activation = "relu")(input_noise) 
    g = tf.keras.layers.Dense(1024, activation = "relu")(input_noise) 
    g = tf.keras.layers.Dense(128*8*8, activation = "relu")(g)
    g = tf.keras.layers.Reshape((8,8,128))(g)
    
    g = tf.keras.layers.Conv2DTranspose(128, kernel_size = (2,2) ,  strides = (2,2) , use_bias = False)(g)
    g = tf.keras.layers.Conv2D( 64  , ( 1 , 1 ) , activation = 'relu' , padding = 'same', name= "block_4")(g) ## 16,16

    g = tf.keras.layers.Conv2DTranspose(32, kernel_size = (2,2) ,  strides = (2,2) , use_bias = False)(g)
    g = tf.keras.layers.Conv2D( 64  , ( 1 , 1 ) , activation = 'relu' , padding = 'same', name = "block_5")(g) ## 32,32
    
    
    if img_shape[0] ==  64:
        g = layers.Conv2DTranspose(32, kernel_size=(2,2) ,  strides=(2,2) , use_bias=False)(g)
        g = layers.Conv2D( 64  , ( 1 , 1 ) , activation='relu' , padding='same', name="block_6")(g) ## 64,64
    
    img = tf.keras.layers.Conv2D( 3 , ( 1 , 1 ) , activation = 'sigmoid' , padding = 'same', name = "final_block")(g) ## 32, 32
    model = tf.keras.models.Model(input_noise, img)
    model.summary() 
    return model
# - - - - - - - - - - - - - - - - - - - - - - 


In [ ]:
#!/usr/python3
# - - - - - - - - - - - - - - - - - - - - - - 
# Load my train image files into a numpy array:
def load_dataset():
    # Loading dataset
    dataset = []

    # Get female faces:
    for dirname, _, filenames in os.walk('../input/celebahq/celeba_hq/train/female'):
        for i, filename, in enumerate(filenames):
            #print(os.path.join(dirname, filename))
            img = tf.compat.v1.keras.preprocessing.image.load_img(os.path.join(dirname, filename), target_size = IMG_SHAPE[:2])
            img = tf.compat.v1.keras.preprocessing.image.img_to_array(img)/255.0
            dataset.append(img)
            if i  == TRAIN_SIZE:
                break
            
    # Get male faces:
    for dirname, _, filenames in os.walk('../input/celebahq/celeba_hq/train/male'):
        for i, filename, in enumerate(filenames):
            #print(os.path.join(dirname, filename))
            img = tf.compat.v1.keras.preprocessing.image.load_img(os.path.join(dirname, filename), target_size = IMG_SHAPE[:2])
            img = tf.compat.v1.keras.preprocessing.image.img_to_array(img)/255.0
            dataset.append(img)
            if i  == TRAIN_SIZE:
                break
    random.shuffle(dataset)
    return np.array(dataset)

#print("Train.shape = {}".format(load_dataset().shape))
# - - - - - - - - - - - - - - - - - - - - - -

# - - - - - - - - - - - - - - - - - - - - - -
# Generate plot of my images:

def plot_images(sample):
    fig = plt.figure(figsize = (30, 10))
    for i in range(1,7):
        ax = fig.add_subplot(1, 7,i)
        ax.imshow(sample[i])
    return plt.show()
# - - - - - - - - - - - - - - - - - - - - - -

def get_noise(nsample=1, nlatent_dim=100):
    noise = np.random.normal(0, 1, (nsample,nlatent_dim))
    return(noise)

# - - - - - - - - - - - - - - - - - - - - - -
# Save network models:
def save_models(generator, discriminator, epoch):
    # If no directory, then create:
    if not os.path.exists(GENERATED_MODEL_PATH):
        os.mkdir(GENERATED_MODEL_PATH)
    # Saving models:
    generator.save(GENERATED_MODEL_PATH+'dcgan_generator_'+str(epoch) + '.h5')
    discriminator.save(GENERATED_MODEL_PATH+'dcgan_discriminator_'+str(epoch) + '.h5')
    print("\n MODELS: GENERATOR + DISCRIMINATOR SAVED!\n")
# - - - - - - - - - - - - - - - - - - - - - -

# - - - - - - - - - - - - - - - - - - - - - -
# Output network training progress:
def show_progress(epoch, d_loss, g_loss):
    print (
            "Epoch {:05.0f} [D loss: {:4.3f}, acc.: {:05.1f}%] [G loss: {:4.3f}]"
            .format(epoch, d_loss[0], 100*d_loss[1], g_loss)
            )
# - - - - - - - - - - - - - - - - - - - - - -

def plot_generated_images(generator,noise,path_save=None,titleadd=""):
    if not os.path.exists(GENERATED_PLOTS_PATH):
        os.mkdir(GENERATED_PLOTS_PATH)
    imgs = generator.predict(noise)
    fig = plt.figure(figsize=(40,10))
    for i, img in enumerate(imgs):
        ax = fig.add_subplot(1,4,i+1)
        ax.imshow(img)
    fig.suptitle("Generated images "+titleadd,fontsize=30)
    
    if path_save is not None:
        plt.savefig(path_save,
                    bbox_inches='tight',
                    pad_inches=0)
        plt.close()
    else:
        plt.show()

In [ ]:
#!/usr/python3

# https://fairyonice.github.io/My-first-GAN-using-CelebA-data.html
# https://www.tensorflow.org/tutorials/generative/dcgan
# https://github.com/vwrs/dcgan-mnist
# - - - - - - - - - - - - - - - - - - - - - -
# My Deep Convolution Generative Adversarial Network:

# - - - - - - - - - - - - - - - - - - - - - -
# DCGAN's model objects:
def dcGan_models():
    #with tpu_strategy.scope():
    generator = build_generator(IMG_SHAPE, noise_shape = NOISE_SHAPE)
    generator.compile(
                loss = 'binary_crossentropy',
                optimizer = OPTIMIZER)

    discriminator  = build_discriminator(IMG_SHAPE)
    discriminator.compile(
                loss = 'binary_crossentropy',
                optimizer = OPTIMIZER,
                metrics   = ['accuracy'])

    noise_inputLayer = tf.keras.layers.Input(shape = NOISE_SHAPE)
    generated_image = generator(noise_inputLayer)
    discriminator.trainable = False
    valid = discriminator(generated_image)

    combined = tf.keras.models.Model(noise_inputLayer, valid)
    combined.compile(
                loss = 'binary_crossentropy',
                optimizer = OPTIMIZER)
    #combined.summary()
    return (combined, discriminator, generator)
# - - - - - - - - - - - - - - - - - - - - - -

# - - - - - - - - - - - - - - - - - - - - - -
# DCGAN network train routine:
def train(models, X_train, noise_plot, RESULTS_PATH="/result/", epochs=10000, batch_size=128):
    combined, discriminator, generator = models
    nlatent_dim = noise_plot.shape[1]
    half_batch  = int(batch_size / 2)
    history = []
    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random half batch of images
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]
        noise = get_noise(half_batch, nlatent_dim)

        # Generate a half batch of new images
        gen_imgs = generator.predict(noise)


        # Train the discriminator q: better to mix them together?
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


        # ---------------------
        #  Train Generator
        # ---------------------

        noise = get_noise(batch_size, nlatent_dim)

        # The generator wants the discriminator to label the generated samples
        # as valid (ones)
        valid_y = (np.array([1] * batch_size)).reshape(batch_size,1)

        # Train the generator
        g_loss = combined.train_on_batch(noise, valid_y)

        history.append({"D":d_loss[0],"G":g_loss})

        # Process progress at each epoch:
        if epoch % 100 == 0:
            show_progress(epoch, d_loss, g_loss)
  
        if epoch % int(epochs/100) == 0:
            plot_generated_images(generator,noise_plot,
                                    path_save=GENERATED_PLOTS_PATH+"plot_{:05.0f}.png".format(epoch),
                                    titleadd="Epoch {}".format(epoch))
        if epoch % 1000 == 0:
            plot_generated_images(generator,noise_plot,
                                    titleadd="Epoch {}".format(epoch))
            save_models(generator, discriminator, str(epoch))
                                              
    #Save final model:
    save_models(generator, discriminator, " FINAL ")
    
    image = combine_images(generator.predict(noise_plot))
    image = image*127.5 + 127.5
    Image.fromarray(image.astype(np.uint8))\
            .save(GENERATED_IMAGE_PATH+"%03.png" % (epoch))
    
    return(history)
# - - - - - - - - - - - - - - - - - - - - - -

def combine_images(generated_images):
    total,width,height = generated_images.shape[:-1]
    cols = int(math.sqrt(total))
    rows = math.ceil(float(total)/cols)
    combined_image = np.zeros((height*rows, width*cols),
                              dtype=generated_images.dtype)

    for index, image in enumerate(generated_images):
        i = int(index/cols)
        j = index % cols
        combined_image[width*i:width*(i+1), height*j:height*(j+1)] = image[:, :, 0]
    return combined_image

def main():
    print("Start")
    X_train = load_dataset()
    noise = get_noise(4, NOISE_SHAPE[0])

    try:
        os.mkdir(RESULTS_PATH)
        os.mkdir(GENERATED_PLOTS_PATH)
        os.mkdir(GENERATED_IMAGE_PATH)
    except:
        pass

    start_time = time.time()
    _models = dcGan_models()
    history = train(_models, X_train, noise, RESULTS_PATH=RESULTS_PATH,epochs=20000, batch_size=128*8)
    end_time = time.time()
    print("-"*10)
    print("Time took: {:4.2f} min".format((end_time - start_time)/60))
    
if __name__ == "__main__":
    main()